# 1. Business Understanding (Phase CRISP-DM)
### Contexte Global :
En Tunisie, la sécurité routière est un enjeu de santé publique majeur. Selon l'Observatoire National de la Sécurité Routière (ONSR), le non-respect du code de la route et le défaut d'équipement de sécurité (comme le casque) sont les causes principales de mortalité. Les méthodes de surveillance manuelles étant limitées, l'automatisation par Intelligence Artificielle devient une nécessité.

### Objectif du Projet Final :
Développer une solution intelligente de surveillance routière capable de :

##### * Identifier les véhicules (Détection de plaques).

##### * Extraire les informations d'immatriculation (OCR des plaques tunisiennes).

##### * Vérifier la conformité sécuritaire (Détection du port du casque).

### Spécificité de ce module :

#### 01_Detection_Plaques.ipynb :
Ce notebook se concentre sur la localisation précise des plaques d'immatriculation sur les véhicules en circulation.

#### 02_OCR_CNN_From_Scratch.ipynb :
Ce notebook est dédié à la lecture optique des caractères (OCR) sur les plaques détectées, en utilisant une architecture CNN conçue "from scratch".

#### 03_Detection_Casques.ipynb :
Ce notebook traite de la détection du port du casque, permettant d'identifier les infractions de sécurité des motards.


In [ ]:
# Installation des librairies manquantes sur Colab
!pip install -q ultralytics
!pip install -q opencv-python-headless

# Importations
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from tqdm import tqdm
import kagglehub
import pathlib

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models


In [ ]:
# 1. Télécharger le dataset spécifique aux casques
path = kagglehub.dataset_download("andrewmvd/helmet-detection")
print("Dataset téléchargé dans :", path)

# 2. Explorer le contenu du dossier racine
print("Contenu du dossier racine :")
print(os.listdir(path))

# 3. Localiser les dossiers d'images et d'annotations
# Dans ce dataset, la structure est : /images et /annotations
image_dir = pathlib.Path(path) / "images"
annot_dir = pathlib.Path(path) / "annotations"

# 4. Vérification et exploration
if image_dir.exists() and annot_dir.exists():
    # Note : ce dataset utilise des fichiers .png
    image_files = list(image_dir.glob("*.png"))
    annot_files = list(annot_dir.glob("*.xml"))

    print(f"\n--- Exploration réussie ---")
    print(f"Nombre d'images trouvées (.png) : {len(image_files)}")
    print(f"Nombre d'annotations trouvées (.xml) : {len(annot_files)}")
    print("Exemples d'images :", [f.name for f in image_files[:3]])
else:
    print("\n[ERREUR] Structure non conforme — exploration manuelle :")
    !ls {path}

Using Colab cache for faster access to the 'helmet-detection' dataset.
Dataset téléchargé dans : /kaggle/input/helmet-detection
Contenu du dossier racine :
['annotations', 'images']

--- Exploration réussie ---
Nombre d'images trouvées (.png) : 764
Nombre d'annotations trouvées (.xml) : 764
Exemples d'images : ['BikesHelmets719.png', 'BikesHelmets219.png', 'BikesHelmets18.png']


# Data Preparation (Extraction des images)

In [ ]:
print("Relancement de l'extraction avec les labels : 'With Helmet' et 'Without Helmet'...")

# On nettoie pour repartir sur de bonnes bases
import shutil
shutil.rmtree(str(class_dir), ignore_errors=True)
for label in ['with_helmet', 'without_helmet']:
    (class_dir / label).mkdir(parents=True, exist_ok=True)

for xml_file in tqdm(annot_files):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    img_name_stem = pathlib.Path(xml_file).stem
    img_path = image_dir / f"{img_name_stem}.png"

    image = cv2.imread(str(img_path))
    if image is None: continue

    for i, obj in enumerate(root.findall('object')):
        raw_label = obj.find('name').text # Ici on récupère 'With Helmet' ou 'Without Helmet'

        # Mapping exact
        if raw_label == 'With Helmet':
            final_label = 'with_helmet'
        elif raw_label == 'Without Helmet':
            final_label = 'without_helmet'
        else:
            continue

        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        crop = image[ymin:ymax, xmin:xmax]
        if crop.size > 0:
            save_name = f"{img_name_stem}_{i}.jpg"
            cv2.imwrite(str(class_dir / final_label / save_name), crop)

print(f"\nExtraction réussie !")
print(f"Images 'With Helmet' : {len(list((class_dir/'with_helmet').glob('*')))}")
print(f"Images 'Without Helmet' : {len(list((class_dir/'without_helmet').glob('*')))}")

Relancement de l'extraction avec les labels : 'With Helmet' et 'Without Helmet'...


100%|██████████| 764/764 [00:13<00:00, 55.85it/s]


Extraction réussie !
Images 'With Helmet' : 952
Images 'Without Helmet' : 482


# 3. Extraction/Cropping

In [ ]:
# Configuration : Normalisation et Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,            # Transformation des pixels de [0-255] à [0-1]
    rotation_range=15,         # Rotation aléatoire de 15 degrés
    width_shift_range=0.1,     # Décalage horizontal
    height_shift_range=0.1,    # Décalage vertical
    horizontal_flip=True,      # Retournement horizontal (miroir)
    validation_split=0.2       # 20% des données réservées pour le test (Validation)
)

# Générateur pour l'entraînement (80% des données)
train_generator = datagen.flow_from_directory(
    class_dir,
    target_size=(224, 224),    # Taille standard pour MobileNetV2
    batch_size=32,
    class_mode='binary',       # Classification binaire (Avec/Sans casque)
    subset='training'
)

# Générateur pour la validation (20% des données)
val_generator = datagen.flow_from_directory(
    class_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 1148 images belonging to 2 classes.
Found 286 images belonging to 2 classes.


# 4. Division du Dataset (Code split-folders)

In [ ]:
# 1. Installer la lib (si pas déjà fait en haut)
!pip install -q split-folders

import splitfolders

# 2. Définir le dossier source (celui où tu as extrait tes images)
# class_dir était : /content/helmet_data
input_folder = "/content/helmet_data"

# 3. Définir le dossier de sortie pour la division
output_folder = "/content/helmet_split"

# 4. Exécuter la division (80% Train, 10% Val, 10% Test)
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.8, .1, .1))

print("Division terminée ! Structure créée dans :", output_folder)

# Vérification
!ls /content/helmet_split

Copying files: 1434 files [00:00, 10521.90 files/s]


Division terminée ! Structure créée dans : /content/helmet_split
test  train  val


# 5. Création des Générateurs de Données

In [ ]:
# Chemins vers les nouveaux dossiers
TRAIN_DIR = "/content/helmet_split/train"
VAL_DIR = "/content/helmet_split/val"
TEST_DIR = "/content/helmet_split/test"

# 1. Augmentation pour le Train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# 2. Simple normalisation pour Val et Test (pas de transformations)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# 3. Création des générateurs
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False # Très important pour l'évaluation finale
)

Found 1146 images belonging to 2 classes.
Found 143 images belonging to 2 classes.
Found 145 images belonging to 2 classes.


# 6. Modeling (Transfer Learning)

In [ ]:
# 1. Charger la base MobileNetV2 pré-entraînée
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# 2. Geler la base pour conserver l'apprentissage initial
base_model.trainable = False

# 3. Ajouter la tête de classification adaptée à notre problème
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # au lieu de flatten pour transformer l'image en un seule vecteur
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),              # Évite le surapprentissage (Overfitting)
    layers.Dense(1, activation='sigmoid') # Sortie : 0 (Pas de casque) ou 1 (Avec casque)
])

# 4. Compilation avec l'optimiseur Adam
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

# 7. Training

In [ ]:
# Lancement de l'entraînement (Vérifie bien que tu as activé le GPU dans Colab !)
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Évaluation sur le dataset de TEST (le juge final)
test_loss, test_acc = model.evaluate(test_generator)
print(f"\nPrécision finale sur le TEST : {test_acc*100:.2f}%")

Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.6310 - loss: 0.7857 - val_accuracy: 0.8042 - val_loss: 0.4572
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 354ms/step - accuracy: 0.7422 - loss: 0.5090 - val_accuracy: 0.8042 - val_loss: 0.4339
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 346ms/step - accuracy: 0.7828 - loss: 0.4494 - val_accuracy: 0.7762 - val_loss: 0.4250
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 345ms/step - accuracy: 0.8114 - loss: 0.4145 - val_accuracy: 0.8112 - val_loss: 0.4021
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 351ms/step - accuracy: 0.8062 - loss: 0.3948 - val_accuracy: 0.8112 - val_loss: 0.4043
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 347ms/step - accuracy: 0.8267 - loss: 0.3844 - val_accuracy: 0.7832 - val_loss: 0.4001
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 346ms/step - accuracy: 0.7989 - loss: 0.3925 - val_accuracy: 0.8322 - val_loss: 0.3800
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 349ms/step - accuracy: 0.8491 - loss: 0.3478 - val_accurac

In [ ]:
# 1. Installer Gradio
!pip install -q gradio

import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

def predict_helmet(img):
    # Prétraitement de l'image
    img = img.resize((224, 224))
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Même normalisation que l'entraînement
    img_array = np.expand_dims(img_array, axis=0) # Ajouter la dimension 'batch'

    # Prédiction
    prediction = model.predict(img_array)[0][0]

    # Interprétation du résultat (Sigmoid : 0=Sans, 1=Avec)
    # On inverse souvent selon l'ordre alphabétique des dossiers
    # with_helmet = 0 ou 1 selon le train_generator.class_indices
    # Vérifions l'ordre :
    labels = {v: k for k, v in train_generator.class_indices.items()}

    if prediction > 0.5:
        res = labels[1].replace("_", " ").title()
        conf = float(prediction)
    else:
        res = labels[0].replace("_", " ").title()
        conf = float(1 - prediction)

    return f"Résultat : {res} (Confiance : {conf:.2%})"

# 2. Créer l'interface
interface = gr.Interface(
    fn=predict_helmet,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Système de Détection de Casque",
    description="Téléchargez une photo d'un conducteur pour vérifier s'il porte un casque."
)

# 3. Lancer l'interface
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://50e734054570e57206.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
